# Create a Medical Assistant RAG System chat with LangChainAgent & ChromaDB,using Together AI API key


[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hosein9574/My-agents/blob/main/Medical_Assistant_Agent.ipynb)


In [ ]:
!pip install -q llama-cpp-python langchain langchain-community langchainhub datasets chromadb sentence-transformers

In [ ]:
from datasets import load_dataset
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA , LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain import hub
from langchain_community.llms import LlamaCpp

import os

In [ ]:
!pip install datasets --upgrade

import pandas as pd
from langchain_community.document_loaders import DataFrameLoader

dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset", split="train")

# Convert the first 100 entries into a pandas DataFrame
df = pd.DataFrame(dataset[:100])

# Load documents using LangChain's DataFrameLoader
df_loader = DataFrameLoader(df, page_content_column="Answer")
df_document = df_loader.load()

These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [ ]:
df[:100]

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
...,...,...,...
95,exams and tests,How to diagnose Parasites - Leishmaniasis ?,Various laboratory methods can be used to diag...
96,treatment,What are the treatments for Parasites - Leishm...,"Before considering treatment, the first step i..."
97,prevention,How to prevent Parasites - Leishmaniasis ?,No vaccines or drugs to prevent infection are ...
98,information,What is (are) Parasites - Toxoplasmosis (Toxop...,A single-celled parasite called Toxoplasma gon...


In [ ]:
# split into chunks
text_splitter = CharacterTextSplitter(chunk_size=1250, separator="\n", chunk_overlap=100)
texts = text_splitter.split_documents(df_document)

In [ ]:
# create embeddings and vector store in chroma
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
chroma_db = Chroma.from_documents(texts, embedding_model, persist_directory="./chromadb")


In [ ]:
# Load model from Together.ai
!pip install -q together

import getpass
from langchain_community.llms import Together

# Prompt user to securely enter their Together.ai API key
os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter your Together.ai API key: ")

# Initialize Together.ai LLM
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.0,
    max_tokens=1024,
    top_p=1,
    together_api_key=os.environ["TOGETHER_API_KEY"],
)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType

# Agent Use cases:

In general, using LangChain agents is useful when there is a need to manage and coordinate multiple tools and processing steps. In summary, the situations where LangChain agents are used include:

- Multi-step responses: When questions or tasks involve multiple steps to solve (e.g., combining medical Q&A with summarizing content).
  
- Utilizing diverse tools: When you want to seamlessly integrate multiple tools like RAG, summarization, or database searches.
  
- Managing dynamic interactions: When answers need to be adjusted based on conversation history and changing conditions.
  
- Coordinating complex processes: In projects that require the agent to autonomously decide which tools to use in order to quickly arrive at comprehensive results.

In [ ]:
# tool1 = RAG
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=chroma_db.as_retriever())

# tool2 = summarizer
summary_prompt = PromptTemplate.from_template("Summarize this text:\n\n{text}")
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

**Rate Limiting and Delay Tool**

This code enforces rate limits on the agent’s requests to avoid exceeding the API limits from **Together.ai**. The system is set to:

*   **1 QPS (Query Per Second): Ensures that only one request is sent every second.**
*   **60 RPM (Requests Per Minute): Limits the agent to no more than 60 requests per minute.**

To achieve this, the code tracks the timestamps of previous requests and enforces a delay whenever necessary:

*   **For QPS: If the previous request was made less than a second ago, the tool waits for the required time before sending the next request.**
*   **For RPM: If 60 requests have been made within the last minute, the tool waits until the minute is over before continuing.**

This ensures compliance with rate limits and prevents errors due to too many requests.

In [ ]:
import time

# Define delayed functions for each tool to respect QPS
def delayed_qa_run(*args, **kwargs):
    time.sleep(1.1)  # sleep 1.1s before running to respect QPS
    return qa.run(*args, **kwargs)

def delayed_summary_run(*args, **kwargs):
    time.sleep(1.1)
    return summary_chain.run(*args, **kwargs)

# Create tools with the delayed functions
tools = [
    Tool(
        name="Medical QA",
        func=delayed_qa_run,
        description="Answering medical questions. Example: What are the symptoms of high blood pressure?"
    ),
    Tool(
        name="Summarizer",
        func=delayed_summary_run,
        description="Summarizing the answer retrived from database or any other text"
    )
]

In [ ]:
# Create agent using `initialize_agent` which handles prompt construction and memory
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # Using a conversational agent for context management
    memory=conversational_memory,
    verbose=True
)

In [ ]:
agent_executor.invoke({"input": """state the symptoms of Lymphocytic Choriomeningitis and summarize the answer."""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical QA
Action Input: What are the symptoms of Lymphocytic Choriomeningitis?
Observation:  The symptoms of Lymphocytic Choriomeningitis (LCMV) usually occur 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain.
Thought: Do I need to summarize the answer? Yes
Action: Summarizer
Action Input: The symptoms of Lymphocytic Choriomeningitis (LCMV) usually occur 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms:

{'input': 'state the symptoms of Lymphocytic Choriomeningitis and summarize the answer.',
 'chat_history': [HumanMessage(content='Is this an important illness?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes, Lymphocytic Choriomeningitis (LCMV) is an important illness. It is a viral infection that can cause inflammation of the membranes surrounding the brain and spinal cord, and can lead to serious complications if left untreated. It is important to seek medical attention if you suspect you may have LCMV, as early treatment can help improve the chances of a good outcome.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Do you know who is Clark Kent?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Clark Kent is a fictional character who is the alter ego of Superman in the DC Comics universe. He is a journalist who works for the Daily Planet and is known for his mild-mannered and unassuming demeanor. In his secret identity

To make queries we simply call the `agent` directly.

First i will try a order not related to the Medical field.

In [ ]:
agent_executor.invoke({"input": """Give me the area of square of 2x2"""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: The area of a square with side length 2 is 2*2 = 4 square units.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [HumanMessage(content='Do you know who is Clark Kent?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Clark Kent is a fictional character who is the alter ego of Superman in the DC Comics universe. He is a journalist who works for the Daily Planet and is known for his mild-mannered and unassuming demeanor. In his secret identity as Superman, he is a powerful superhero with extraordinary abilities such as super strength, speed, invulnerability, and flight. Clark Kent is often portrayed as being in love with Lois Lane, a fellow journalist at the Daily Planet.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square with side length 2 is 2*2 = 4 square units.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Give me the area of square of 2x2', additio

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [ ]:
agent_executor.invoke({"input": """Do you know who is Clark Kent?"""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Clark Kent is a fictional character who is the alter ego of Superman in the DC Comics universe. He is a journalist who works for the Daily Planet and is known for his mild-mannered and unassuming demeanor. In his secret identity as Superman, he is a powerful superhero with extraordinary abilities, including super strength, speed, invulnerability, and flight. Clark Kent is often portrayed as being in love with Lois Lane, a fellow journalist at the Daily Planet. The character of Clark Kent was created by Jerry Siegel and Joe Shuster, and first appeared in the comic book Action Comics #1 in 1938.

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square with side length 2 is 2*2 = 4 square units.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square with side length 2 is 2*2 = 4 square units.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='state the symptoms of Lymphocytic Choriomeningitis and summarize the answer.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The symptoms of Lymphocytic Choriomeningitis (LCMV) usually include fever, malaise, lack of appetite, muscle aches, headache, nausea, vomiting, sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain. The second phase of the illness is characterized by a r

It has not accessed either, as the model has been able to identify that it is not a question related to the database that LangChain provides.

Now it's time to try with a question related to Medicine. Let's see if the model can understand that it should first look for information in the vector database at its disposal.

In [ ]:
 agent_executor.memory.clear()

In [ ]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,how can I confirm the diagnosis?"""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical QA
Action Input: How can I confirm the diagnosis of Botulism?
Observation:  You can confirm the diagnosis of botulism by performing special tests such as a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Additionally, tests for botulinum toxin and for bacteria that cause botulism can be performed at some state health department laboratories and at CDC.
Thought: Do I need to use a tool? No
AI: To confirm the diagnosis of Botulism, you can perform special tests such as a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Additionally, tests for botulinum toxin and for bacteria that cause botulism can be performed at some state health department laboratories and at CDC.

> Finished chain.


{'input': 'I have a patient that can have Botulism,how can I confirm the diagnosis?',
 'chat_history': [],
 'output': 'To confirm the diagnosis of Botulism, you can perform special tests such as a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Additionally, tests for botulinum toxin and for bacteria that cause botulism can be performed at some state health department laboratories and at CDC.'}

Perfect, the most important thing for us is that it has been able to identify that it should go to the medical database to search for information about the symptoms.

In [ ]:
agent_executor.invoke({"input": """Is this an important illness?"""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Yes, Botulism is a serious and potentially life-threatening illness. It is caused by a neurotoxin produced by the bacterium Clostridium botulinum, and it can cause paralysis of the muscles that control breathing, swallowing, and speaking. If left untreated, it can be fatal. It is important to diagnose and treat Botulism as soon as possible to prevent serious complications.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,how can I confirm the diagnosis?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='To confirm the diagnosis of Botulism, you can perform special tests such as a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Additionally, tests for botulinum toxin and for bacteria that cause botulism can be performed at some state health department laboratories and at CDC.', additional_kwargs={}, response_metadata={})],
 'output': 'Yes, Botulism is a serious and potentially life-threatening illness. It is caused by a neurotoxin produced by the bacterium Clostridium botulinum, and it can cause paralysis of the muscles that control breathing, swallowing, and speaking. If left untreated, it can be fatal. It is important to diagnose and treat Botulism as soon as possible to prevent ser